#Importação de Bibliotecas

In [9]:
# mlflow é usado para rastrear experimentos, registrar e servir modelos de machine learning
import mlflow

# mlflow.sklearn permite integração direta entre modelos do scikit-learn e o MLflow (para logar e salvar modelos)
import mlflow.sklearn

# matplotlib.pyplot é usado para criar gráficos e visualizações básicas (como curva ROC ou matriz de confusão)
import matplotlib.pyplot as plt

# GaussianNB é o classificador Naive Bayes para variáveis contínuas baseado em distribuição Gaussiana
from sklearn.naive_bayes import GaussianNB

# Métricas de avaliação para modelos de classificação (ex: acurácia, recall, f1-score, etc.)
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score,
    roc_auc_score, log_loss, confusion_matrix, roc_curve
)

# Importa a função train_test_split do scikit.learn
from sklearn.model_selection import train_test_split

# Now use the function
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores, classe,
                                        test_size=0.3, random_state=123) #Aqui, dividimos os dados entre treino (70%) e teste (30%) e aplicamos uma semente "radom_state=123"
                                                                        #para garantir a reprodutibilidade do experimento
# seaborn é uma biblioteca de visualização que facilita a criação de gráficos estatísticos mais bonitos (como heatmaps)
import seaborn as sns

# pandas é usado para leitura, manipulação e análise de dados em formato de tabelas (DataFrames)
import pandas as pd

# numpy é usado para manipulação eficiente de arrays e operações numéricas
import numpy as np

#Esses pacotes cobrem todo o pipeline: modelagem, avaliação, visualização e registro de modelo com o MLflow.

#Carregando e visualizando a base de dados

In [2]:
credito = pd.read_csv('Credit.csv')
credito.shape

(1000, 21)

In [3]:
credito.head(5) #Visualiza as 5 primeiras linhas 

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


#Convertendo variáveis string em variáveis numéricas

In [4]:
# Loop que percorre todas as colunas do DataFrame
for col in credito.columns:  # 'col' vai assumir cada nome de coluna, um por um

    # Verifica se a coluna contém dados do tipo texto (string)
    if credito[col].dtype == 'object':  # Isso evita mexer em colunas numéricas

        # Converte a coluna para o tipo categórico e atribui códigos numéricos a cada categoria
        credito[col] = credito[col].astype('category').cat.codes

        # .astype('category') -> transforma o texto em categorias (internamente)
        # .cat.codes -> atribui um número inteiro para cada categoria


In [5]:
credito.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0


#Definindo os previsores e a variável target

In [6]:
previsores = credito.iloc[:, 0:20].values
# seleciona todas as linhas (:) e as colunas de 0 até 19 (0:20, o 20 é exclusivo)
# .values transforma em um array NumPy, que é o formato esperado para muitos modelos ML
# 'previsores' são as variáveis independentes (features) que o modelo vai usar para prever

classe = credito.iloc[:, 20].values
# seleciona todas as linhas (:) da coluna de índice 20 (a 21ª coluna)
# essa coluna é a variável dependente (target, label) que o modelo deve aprender a prever
# também transformada em array NumPy


In [7]:
previsores #exibir os previsores em formato array Numpy

array([[ 2,  6,  1, ...,  1,  1,  1],
       [ 1, 48,  3, ...,  1,  0,  1],
       [ 0, 12,  1, ...,  2,  0,  1],
       ...,
       [ 0, 12,  3, ...,  1,  0,  1],
       [ 2, 45,  3, ...,  1,  1,  1],
       [ 1, 45,  1, ...,  1,  0,  1]])

#Divisão dos dados entre treino e teste

In [10]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,classe,
                                        test_size=0.3,random_state=123) #Aqui, dividimos os dados entre treino (70%) e teste (30%) e aplicamos uma semente "radom_state=123"
                                                                        #para garantir a reprodutibilidade do experimento

#Implementação de um experimento Naive Bayes com hiperparâmetros

In [11]:
mlflow.set_experiment("nbexperimento") #Configura o experimento e dá um nome a ele

# Lista com diferentes valores de smoothing para testar o impacto desse hiperparâmetro no modelo Naive Bayes
valores_smoothing = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]

for smoothing in valores_smoothing:
    with mlflow.start_run(): #Inicia o experimento com um laço
        # Treinamento
        naive_bayes = GaussianNB(var_smoothing=smoothing)
        naive_bayes.fit(X_treinamento, y_treinamento)
        previsoes = naive_bayes.predict(X_teste)
        probs = naive_bayes.predict_proba(X_teste)[:, 1]  # Probabilidades para ROC

        # Calculando as Métricas
        acuracia = accuracy_score(y_teste, previsoes)
        recall = recall_score(y_teste, previsoes)
        precision = precision_score(y_teste, previsoes)
        f1 = f1_score(y_teste, previsoes)
        auc = roc_auc_score(y_teste, previsoes)
        log = log_loss(y_teste, previsoes)

        # Registrando hiperparâmetro e métricas
        mlflow.log_param("var_smoothing", smoothing)
        mlflow.log_metric("acuracia", acuracia)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1", f1)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("log", log)

        # Calculando e Plotando a Matriz de Confusão
        cm = confusion_matrix(y_teste, previsoes)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f"Confusion Matrix - smoothing={smoothing}")
        plt.xlabel("Previsto")
        plt.ylabel("Real")
        conf_file = f"confusion_{smoothing}.png"
        plt.savefig(conf_file)
        mlflow.log_artifact(conf_file)
        plt.close()

        # Curva ROC
        fpr, tpr, _ = roc_curve(y_teste, probs)
        plt.figure(figsize=(6, 4))
        plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
        plt.xlabel("FPR (Falsos Positivos)")
        plt.ylabel("TPR (Verdadeiros Positivos)")
        plt.title(f"ROC Curve - smoothing={smoothing}")
        plt.legend()
        roc_file = f"roc_{smoothing}.png"
        plt.savefig(roc_file)
        mlflow.log_artifact(roc_file)
        plt.close()

        # Registrando o modelo
        mlflow.sklearn.log_model(naive_bayes, name="ModeloNB")

        # Print do ID da execução do modelo
        print(f"Run finalizada com var_smoothing={smoothing}. ID:", mlflow.active_run().info.run_id)

    mlflow.end_run() #Finaliza o experimento

2025/07/01 18:23:08 INFO mlflow.tracking.fluent: Experiment with name 'nbexperimento' does not exist. Creating a new experiment.
2025/07/01 18:23:08 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export G

Run finalizada com var_smoothing=1e-09. ID: e16fafc09c00418f9ec859ca3b1f4151


2025/07/01 18:23:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizada com var_smoothing=1e-08. ID: 5c90561ce2944c348a582bb16bbe9a81


2025/07/01 18:23:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizada com var_smoothing=1e-07. ID: a8d928b7d3f2459ca316ff151a6679b9


2025/07/01 18:23:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizada com var_smoothing=1e-06. ID: 10f7dc5efc21451dbd0d13c10936ca21


2025/07/01 18:23:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizada com var_smoothing=1e-05. ID: 8e2374efc95b4009bac73d8118d5c7c0


In [ ]:
#Para ver os modelos recém-treinados no Mlflow, vá até o prompt do anaconda e digite:

#(base) C:\Users\Seu_Usuario>mlflow ui

#Após isso, vai aparecer o seguinte resultado: 
# INFO:waitress:Serving on http://127.0.0.1:5000

#A partir disso, você vai precisar clicar no link http usando ctrl + enter para acessar a página do Mlflow com os experimentos já rodados e versionados.